In [1]:
import os
import cv2
import numpy as np

Defining preprocessing function

In [2]:

def preprocess_image(image_path,target_size=(224,224)):
    image=cv2.imread(image_path)
    image=cv2.resize(image,target_size)
    image=image.astype(np.float32) / 255.0
    return image

Loading and preprocessing the Image

In [3]:

image_dir='./Cardiac_dataset'
image_paths=[os.path.join(image_dir,filename) for filename in os.listdir(image_dir) if filename.endswith('.jpg')]
preprocessed_images=[]
labels=[]
for image_path in image_paths:
    preprocessed_images.append(preprocess_image(image_path))
    if "Normal" in image_path:
        labels.append(0)  # Assigning the label 0 for "healthy" images
    else:
        labels.append(1)  # Assigning the label 1 for "diseased" images
X=np.array(preprocessed_images)
y=np.array(labels)

Data Splitting

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

Training Our model with features like max pooling, fully connected neural networks and learning rate sheduling

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4,decay_steps=100,decay_rate=0.9)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_scheduler),loss='binary_crossentropy',metrics=['accuracy'])
early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,restore_best_weights=True)
num_epochs = 100
model.fit(X_train,y_train,epochs=num_epochs,validation_data=(X_val,y_val),callbacks=[early_stopping])
test_loss,test_accuracy = model.evaluate(X_test,y_test)
print(f'Test accuracy: {test_accuracy}')
predictions = model.predict(X_test)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 11s 493ms/step - accuracy: 0.6765 - loss: 0.6940 - val_accuracy: 0.7194 - val_loss: 0.5893
Epoch 2/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 422ms/step - accuracy: 0.6611 - loss: 0.6429 - val_accuracy: 0.7194 - val_loss: 0.5745
Epoch 3/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 408ms/step - accuracy: 0.6864 - loss: 0.5848 - val_accuracy: 0.7194 - val_loss: 0.5285
Epoch 4/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 396ms/step - accuracy: 0.7109 - loss: 0.5179 - val_accuracy: 0.7338 - val_loss: 0.4810
Epoch 5/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 402ms/step - accuracy: 0.7116 - loss: 0.4704 - val_accuracy: 0.7410 - val_loss: 0.4278
Epoch 6/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 404ms/step - accuracy: 0.7993 - loss: 0.4068 - val_accuracy: 0.8849 - val_loss: 0.3685
Epoch 7/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 419ms/step - accuracy: 0.9026 - loss: 0.3330 - val_accuracy: 0.8849 - val_loss: 0.3183
Epoch 8/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 420ms/step - accuracy: 0.9085 - loss: 0.2789 - val_acc

In [6]:
model.save("./model/heard_disease_predictor.h5")